In [1]:
include("../MyReverseDiff.jl")
include("../MyEmbedding.jl")
include("../MyMlp.jl")
include("../TensorOperations.jl")

using .MyReverseDiff
using .MyMlp
using .TensorOperations
using JLD2
using Printf
using Random
using Flux
using LinearAlgebra

# Convolution

In [61]:
xo = reshape([2.0f0 3.0f0 1.0f0 4.0f0 7.0f0],5,1,1)#randn(Float32, 2, 1, 1)
m = reshape([1.0f0 2.5f0],2, 1, 1)#randn(Float32, 2, 1, 1)
mm = reverse(reshape(m,2,1))
g = diagm([1.0f0, 2.0f0, 3.0f0, 4.0f0, 1.0f0])*ones(Float32, size(xo, 1), size(xo, 2) * size(m, 2), size(xo, 3))


#@btime MyReverseDiff.dif_convolution(x, m, g)
ym = MyReverseDiff.multi_convolution(xo,mm)
yf = Flux.conv(xo,m)
dm = MyReverseDiff.dif_convolution(xo,mm,g)

(Float32[2.5; 6.0; … ; 13.0; 6.5;;;], Float32[34.0; 45.0;;])

In [66]:
MyReverseDiff.multi_convolution(xo,g[:,:,1])

5×1×1 Array{Float32, 3}:
[:, :, 1] =
 34.0
 45.0
 30.0
 18.0
  7.0

In [63]:
MyReverseDiff.multi_convolution(reverse(xo),g[:,:,1])

5×1×1 Array{Float32, 3}:
[:, :, 1] =
 32.0
 23.0
 13.0
  7.0
  2.0

In [ ]:
using Flux
using Flux: Conv, params
using Random

scale = diagm([1.0f0, 2.0f0, 3.0f0, 4.0f0])

# 1. Dane wejściowe (obraz) i warstwa konwolucyjna
Random.seed!(123)

x = xo#rand(Float32, 5, 5, 1, 1)  # Obraz: 5×5, 1 kanał, batch size 1

conv = Conv((2, ), 1 => 1, stride=1, pad=0)
conv.weight .= m

# 2. Gradient względem wejścia (∂L/∂x)
grad_input = gradient(x -> sum(scale*reshape(conv(x),4,1)), x)[1]

# 3. Gradient względem wag i biasów (∂L/∂W, ∂L/∂b)
loss_fn() = sum(scale*reshape(conv(x),4,1))
grads = gradient(loss_fn, params(conv))
grad_weights = grads[conv.weight]
grad_bias    = grads[conv.bias]

# 4. Wyświetlenie wyników
println("Gradient względem wejścia (∂L/∂x):")
println(grad_input)

println("\nGradient względem maski (∂L/∂W):")
println(grad_weights)

println("\nGradient względem biasu (∂L/∂b):")
println(grad_bias)

println(scale*reshape(conv(x),4,1))


Gradient względem wejścia (∂L/∂x):
Float32[2.5; 6.0; 9.5; 13.0; 4.0;;;]

Gradient względem maski (∂L/∂W):
Float32[45.0; 27.0;;;]

Gradient względem biasu (∂L/∂b):
Float32[10.0]
Float32[8.0; 17.0; 19.5; 68.0;;]


┌ Warning: Implicit gradients such as `gradient(f, ::Params)` are deprecated in Flux!
│ Please see the docs for new explicit form.
│   caller = top-level scope at jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:21
└ @ Core c:\Users\RetailAdmin\Desktop\AWiD\MyMlp\src\notebooks\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:21


# Dense3D

In [3]:
x = reshape([1.0f0 2.0f0 3.0f0 2.0f0 3.0f0 4.0f0 4.0f0 5.0f0 6.0f0 5.0f0 6.0f0 7.0f0],2,3,2)
A = [6.0f0 9.0f0; 4.0f0 2.0f0]
b = Matrix([1.0f0 2.0f0]')
y = MyReverseDiff.dense3Dfun(x,A,b)
g = ones(Float32, size(y))
d = MyReverseDiff.dif_dense3Dfun(x,A,b,g)

(Float32[10.0 10.0 10.0; 11.0 11.0 11.0;;; 10.0 10.0 10.0; 11.0 11.0 11.0], Float32[11.5 12.5; 11.5 12.5], Float32[3.0; 3.0;;])

In [9]:
id = 1
db = sum(g[:,:,id], dims=2)
dA1 = g[:,:,1] * x[:,:,1]'
dA2 = g[:,:,2] * x[:,:,2]'

2×2 Matrix{Float32}:
 16.0  17.0
 16.0  17.0

# Flatten

In [13]:
x = reshape([6.0f0 7.0f0 1.0f0 2.0f0 3.0f0 4.0f0 2.0f0 3.0f0 4.0f0 5.0f0 6.0f0 5.0f0],2,3,2)
println(x)
# forward
y = reshape(x,size(x,1)*size(x,2),size(x,3))
# backward
println(reshape(y,size(x,1),size(x,2),size(x,3)))

Float32[6.0 1.0 3.0; 7.0 2.0 4.0;;; 2.0 4.0 6.0; 3.0 5.0 5.0]
Float32[6.0 1.0 3.0; 7.0 2.0 4.0;;; 2.0 4.0 6.0; 3.0 5.0 5.0]
